In [1]:
import json


with open("../data/speeches_lemmatized.json") as f:
    speeches = json.load(f)

# Preprocessing the corpus

In [2]:
from collections import defaultdict


def group_speeches_by_quarter_century(speeches_list):
    """Group speeches into quarter centuries and create text files.

    Args:
        speeches_list: List of dictionaries containing speech data

    Returns:
        Dictionary with quarter century ranges as keys and filenames as values

    """
    # Group speeches by quarter century
    quarter_groups = defaultdict(list)

    for speech in speeches_list:
        year = int(speech["date"][:4])  # Get year from date
        # Calculate quarter century (1800-1824, 1825-1849, etc.)
        quarter_start = year - (year % 25)
        quarter_end = quarter_start + 24
        quarter_key = f"{quarter_start}-{quarter_end}"

        quarter_groups[quarter_key].append(speech)

    # Create text files for each quarter century
    file_paths = {}
    for quarter, speeches in quarter_groups.items():
        # Sort speeches by date
        speeches.sort(key=lambda x: x["date"])

        # Create lemmatized version
        lemma_text = "\n".join(s["lemmatized"] for s in speeches)
        lemma_filename = f"../data/speeches_{quarter}_lemmatized.txt"
        with open(lemma_filename, "w", encoding="utf-8") as f:
            f.write(lemma_text)

        # Create transcript version
        transcript_text = "\n".join(s["transcript"] for s in speeches)
        transcript_filename = f"../data/speeches_{quarter}_transcript.txt"
        with open(transcript_filename, "w", encoding="utf-8") as f:
            f.write(transcript_text)

        file_paths[quarter] = {
            "lemmatized": lemma_filename,
            "transcript": transcript_filename,
            "count": len(speeches),
        }

    # Print summary
    print("Quarter Century Statistics:")
    for quarter, info in file_paths.items():
        print(f"{quarter}: {info['count']} speeches")

    return file_paths


file_paths = group_speeches_by_quarter_century(speeches)

Quarter Century Statistics:
1800-1824: 59 speeches
1900-1924: 91 speeches
1975-1999: 150 speeches
1825-1849: 90 speeches
2000-2024: 164 speeches
1950-1974: 165 speeches
1925-1949: 103 speeches
1875-1899: 99 speeches
1775-1799: 28 speeches
1850-1874: 108 speeches
2025-2049: 2 speeches


# Building language models

In [3]:
from pathlib import Path

from chronowords.algebra.svd import SVDAlgebra


def create_embeddings_for_periods(file_paths, use_lemmatized=True):
    """Create word embeddings for each quarter-century period.

    Args:
        file_paths: Dictionary from group_speeches_by_quarter_century
        use_lemmatized: Whether to use lemmatized or transcript texts

    Returns:
        Dictionary of {period: SVDAlgebra model}

    """
    models = {}

    for period, info in file_paths.items():
        print(f"\nProcessing period {period}")

        # Choose which text version to use
        filename = info["lemmatized" if use_lemmatized else "transcript"]

        # Create generator for the corpus
        def read_corpus():
            with open(filename, encoding="utf-8") as f:
                for line in f:
                    if line.strip():  # Skip empty lines
                        yield line.strip().lower()  # Lowercase everything

        # Initialize and train model
        model = SVDAlgebra(
            n_components=100,  # Smaller dimension for historical texts
            window_size=5,
            min_word_length=3,
            cms_width=1_000_000,  # 1M width should be enough for this corpus
            cms_depth=5,
        )

        try:
            # Train model
            # print(f"Training model for {period}...")
            model.train(read_corpus())

            # Print some statistics
            # print(f"Vocabulary size: {len(model.vocabulary)}")
            # print(f"Sample words: {model.vocabulary[:10]}")

            models[period] = model
        except Exception as e:
            print(f"Error processing period {period}: {e!s}")

    return models


# Create models for both lemmatized and transcript versions
print("Creating models for lemmatized texts...")
lemma_models = create_embeddings_for_periods(file_paths, use_lemmatized=True)

print("\nCreating models for transcript texts...")
transcript_models = create_embeddings_for_periods(file_paths, use_lemmatized=False)

# Save models for later use
for period, model in lemma_models.items():
    save_path = Path(f"../models/lemmatized/{period}")
    save_path.mkdir(parents=True, exist_ok=True)
    model.save_model(save_path)

for period, model in transcript_models.items():
    save_path = Path(f"../models/transcript/{period}")
    save_path.mkdir(parents=True, exist_ok=True)
    model.save_model(save_path)

Creating models for lemmatized texts...

Processing period 1800-1824
Counting words and skipgrams...
Building vocabulary...
Final embeddings shape: (1331, 100)
Embeddings non-zeros: 133100
Min norm: 1.7154927366809808e-05

Processing period 1900-1924
Counting words and skipgrams...
Building vocabulary...
Final embeddings shape: (1383, 100)
Embeddings non-zeros: 138300
Min norm: 1.7672577773045495e-05

Processing period 1975-1999
Counting words and skipgrams...
Building vocabulary...
Final embeddings shape: (1190, 100)
Embeddings non-zeros: 119000
Min norm: 1.8160011020070682e-05

Processing period 1825-1849
Counting words and skipgrams...
Building vocabulary...
Final embeddings shape: (1298, 100)
Embeddings non-zeros: 129800
Min norm: 1.7878387962087844e-05

Processing period 2000-2024
Counting words and skipgrams...
Building vocabulary...
Final embeddings shape: (1188, 100)
Embeddings non-zeros: 118800
Min norm: 1.802807329202471e-05

Processing period 1950-1974
Counting words and ski

# Semantic shift

In [4]:
from pathlib import Path


def load_and_sort_models(base_path="../models/lemmatized"):
    models = {}
    for period_path in Path(base_path).iterdir():
        if period_path.is_dir():
            period = period_path.name
            loaded_model = SVDAlgebra()
            loaded_model.load_model(period_path)
            models[period] = loaded_model

    # Sort by start year
    sorted_periods = sorted(models.keys(), key=lambda x: int(x.split("-")[0]))
    return {period: models[period] for period in sorted_periods}


models = load_and_sort_models()
print("Periods in chronological order:", list(models.keys()))

target_words = [
    "freedom",
    "democracy",
    "constitution",
    "justice",
    "government",
    "power",
    "law",
    "authority",
    "america",
    "union",
    "state",
    "nation",
]

Periods in chronological order: ['1775-1799', '1800-1824', '1825-1849', '1850-1874', '1875-1899', '1900-1924', '1925-1949', '1950-1974', '1975-1999', '2000-2024', '2025-2049']


In [5]:
from chronowords.alignment.procrustes import ProcustesAligner


def analyze_shifts(models, target_words=None):
    """Analyze semantic shifts between consecutive periods."""
    periods = sorted(models.keys())
    results = {}

    for i in range(len(periods) - 1):
        period1, period2 = periods[i], periods[i + 1]
        model1, model2 = models[period1], models[period2]

        # Align embeddings
        aligner = ProcustesAligner()
        metrics = aligner.fit(
            model1.embeddings,
            model2.embeddings,
            model1.vocabulary,
            model2.vocabulary,
        )

        print(f"\nAligned {period1} -> {period2}")
        print(f"Aligned words: {metrics.num_aligned_words}")
        print(f"Average similarity: {metrics.average_cosine_similarity:.3f}")

        # Analyze specific words
        if target_words:
            shifts = []
            for word in target_words:
                sim = aligner.get_word_similarity(
                    word, model1.embeddings, model2.embeddings
                )
                if sim is not None:
                    shifts.append((word, 1 - sim))  # Convert to distance

            # Sort by shift magnitude
            shifts.sort(key=lambda x: x[1], reverse=True)
            results[f"{period1}->{period2}"] = shifts

            print("\nTop shifted words:")
            for word, shift in shifts[:5]:
                print(f"{word}: {shift:.3f}")

    return results


# Analyze key political concepts
target_words = [
    "freedom",
    "democracy",
    "government",
    "power",
    "war",
    "peace",
    "america",
    "union",
    "state",
    "constitution",
    "rights",
    "justice",
    "law",
]

shifts = analyze_shifts(models, target_words)


Aligned 1775-1799 -> 1800-1824
Aligned words: 727
Average similarity: 0.318

Top shifted words:
freedom: 1.026
union: 0.772
power: 0.765
constitution: 0.725
law: 0.708

Aligned 1800-1824 -> 1825-1849
Aligned words: 791
Average similarity: 0.301

Top shifted words:
state: 0.809
union: 0.797
war: 0.785
freedom: 0.720
law: 0.720

Aligned 1825-1849 -> 1850-1874
Aligned words: 825
Average similarity: 0.299

Top shifted words:
union: 0.774
constitution: 0.765
war: 0.751
power: 0.749
government: 0.742

Aligned 1850-1874 -> 1875-1899
Aligned words: 780
Average similarity: 0.304

Top shifted words:
freedom: 1.143
state: 0.866
law: 0.826
peace: 0.799
government: 0.757

Aligned 1875-1899 -> 1900-1924
Aligned words: 781
Average similarity: 0.305

Top shifted words:
freedom: 0.954
war: 0.849
state: 0.782
constitution: 0.728
peace: 0.711

Aligned 1900-1924 -> 1925-1949
Aligned words: 781
Average similarity: 0.302

Top shifted words:
democracy: 1.081
law: 0.911
state: 0.796
power: 0.771
union: 0.756

In [6]:
from pathlib import Path

import altair as alt
import pandas as pd


# Load preprocessed models
models = load_and_sort_models()
periods = sorted(models.keys(), key=lambda x: int(x.split("-")[0]))

shift_data = []
# For each period transition and its shifts
for i in range(len(periods) - 1):
    period1, period2 = periods[i], periods[i + 1]
    period_pair = f"{period1} → {period2}"

    # Get the shift data for this period transition
    model1, model2 = models[period1], models[period2]
    aligner = ProcustesAligner()
    metrics = aligner.fit(
        model1.embeddings,
        model2.embeddings,
        model1.vocabulary,
        model2.vocabulary,
    )

    # Calculate shifts for target words
    for word in target_words:
        sim = aligner.get_word_similarity(word, model1.embeddings, model2.embeddings)
        if sim is not None:
            shift_data.append(
                {
                    "period": period_pair,
                    "word": word,
                    "shift": 1 - sim,  # Convert similarity to distance
                    "year": int(period1.split("-")[0]),
                }
            )

# Create DataFrame
df = pd.DataFrame(shift_data)

timeline = (
    alt.Chart(df)
    .mark_line(point=True)
    .encode(
        x=alt.X("year:Q", title="Year"),
        y=alt.Y("shift:Q", title="Semantic Change"),
        color="word:N",
        tooltip=["word", "year", "shift"],
    )
    .properties(width=800, height=400, title="Semantic Shifts Over Time")
    .interactive()
)

# Display the visualization
timeline

alt.Chart(...)

In [7]:
df

,period,word,shift,year
0,1775-1799 → 1800-1824,freedom,1.026043,1775
1,1775-1799 → 1800-1824,government,0.699798,1775
2,1775-1799 → 1800-1824,power,0.765479,1775
3,1775-1799 → 1800-1824,war,0.555635,1775
4,1775-1799 → 1800-1824,peace,0.663436,1775
...,...,...,...,...
109,2000-2024 → 2025-2049,america,0.799971,2000
110,2000-2024 → 2025-2049,state,0.660920,2000
111,2000-2024 → 2025-2049,constitution,0.704149,2000
112,2000-2024 → 2025-2049,justice,0.655690,2000


In [8]:
shift_data = []
# Use first period as reference
base_period = periods[0]
base_model = models[base_period]

for current_period in periods[1:]:
    current_model = models[current_period]
    aligner = ProcustesAligner()
    metrics = aligner.fit(
        base_model.embeddings,
        current_model.embeddings,
        base_model.vocabulary,
        current_model.vocabulary,
    )

    for word in target_words:
        sim = aligner.get_word_similarity(
            word, base_model.embeddings, current_model.embeddings
        )
        if sim is not None:
            shift_data.append(
                {"period": current_period, "word": word, "cumulative_shift": 1 - sim}
            )

df_cumulative = pd.DataFrame(shift_data)
df_cumulative["year"] = df_cumulative["period"].apply(lambda x: int(x.split("-")[0]))

cumulative_timeline = (
    alt.Chart(df_cumulative)
    .mark_line(point=True)
    .encode(
        x=alt.X("year:Q", title="Year"),
        y=alt.Y("cumulative_shift:Q", title="Cumulative Semantic Change from 1775"),
        color="word:N",
        tooltip=["word", "year", "cumulative_shift"],
    )
    .properties(
        width=800, height=400, title="Cumulative Semantic Shifts from First Period"
    )
    .interactive()
)

cumulative_timeline

alt.Chart(...)

In [9]:
df_cumulative

,period,word,cumulative_shift,year
0,1800-1824,freedom,1.026043,1800
1,1800-1824,government,0.699798,1800
2,1800-1824,power,0.765479,1800
3,1800-1824,war,0.555635,1800
4,1800-1824,peace,0.663436,1800
...,...,...,...,...
104,2025-2049,america,0.722062,2025
105,2025-2049,state,0.577215,2025
106,2025-2049,constitution,0.432402,2025
107,2025-2049,justice,0.613866,2025


# Topic modeling with NMF

In [10]:
from chronowords.topics.nmf import TopicModel


def analyze_topics_by_period(models, n_topics=10):
    """Create topic models for each time period using existing PPMI matrices.

    Args:
        models: Dictionary of period -> model data from load_and_sort_models()
        n_topics: Number of topics to extract

    """
    topic_models = {}

    for period, model_data in models.items():
        print(f"\nAnalyzing topics for {period}")

        # Get embeddings and vocabulary
        embeddings = model_data.M_dense
        embeddings[embeddings < 0.0] = 0.0  # quick fix for negative values
        vocabulary = model_data.vocabulary

        # Create topic model on existing embeddings
        topic_model = TopicModel(n_topics=n_topics)
        topic_model.fit(embeddings, vocabulary)

        # Print topics
        print("\nTop topics:")
        topic_model.print_topics()

        topic_models[period] = topic_model

    return topic_models


# Create topic models using existing data
topic_models = analyze_topics_by_period(models)


Analyzing topics for 1775-1799


/Users/varjuzoltan/Library/Caches/pypoetry/virtualenvs/chronowords-UWSi7A_J-py3.12/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(



Top topics:

Topic 0:
  inconvenience: 0.0026
  over: 0.0026
  take: 0.0025
  amount: 0.0025
  obey: 0.0025
  judiciary: 0.0025
  progressive: 0.0024
  embarrassment: 0.0024
  remove: 0.0024
  particular: 0.0023

Topic 1:
  history: 0.0034
  punish: 0.0029
  lay: 0.0028
  make: 0.0024
  inform: 0.0023
  accommodation: 0.0023
  subscribe: 0.0023
  compel: 0.0022
  grant: 0.0022
  weaken: 0.0022

Topic 2:
  executive: 0.0029
  election: 0.0029
  domestic: 0.0029
  month: 0.0028
  seem: 0.0026
  humble: 0.0026
  commission: 0.0026
  conviction: 0.0025
  impose: 0.0025
  public: 0.0024

Topic 3:
  maxim: 0.0031
  population: 0.0030
  secure: 0.0027
  retirement: 0.0027
  adams: 0.0027
  john: 0.0025
  fund: 0.0025
  capture: 0.0024
  disclose: 0.0023
  consistent: 0.0023

Topic 4:
  competent: 0.0034
  wait: 0.0030
  way: 0.0029
  all: 0.0028
  moderation: 0.0026
  shew: 0.0026
  more: 0.0025
  authorize: 0.0024
  habit: 0.0023
  states: 0.0023

Topic 5:
  friendly: 0.0031
  amicable: 0.0

/Users/varjuzoltan/Library/Caches/pypoetry/virtualenvs/chronowords-UWSi7A_J-py3.12/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(



Top topics:

Topic 0:
  private: 0.0029
  rise: 0.0029
  restriction: 0.0029
  peculiarly: 0.0028
  destiny: 0.0026
  present: 0.0026
  late: 0.0026
  proclamation: 0.0026
  piracy: 0.0026
  water: 0.0025

Topic 1:
  brother: 0.0029
  precede: 0.0029
  involve: 0.0028
  rule: 0.0027
  similar: 0.0027
  nevertheless: 0.0027
  rely: 0.0026
  conviction: 0.0025
  sudden: 0.0025
  ocean: 0.0025

Topic 2:
  supply: 0.0029
  give: 0.0028
  defect: 0.0028
  erect: 0.0027
  prayer: 0.0026
  proof: 0.0026
  essentially: 0.0026
  defense: 0.0025
  execute: 0.0025
  intention: 0.0025

Topic 3:
  neither: 0.0030
  hope: 0.0030
  taxis: 0.0030
  desire: 0.0030
  public: 0.0028
  proposition: 0.0027
  home: 0.0026
  capital: 0.0025
  behold: 0.0025
  resolution: 0.0024

Topic 4:
  foundation: 0.0031
  importance: 0.0029
  regulate: 0.0027
  witness: 0.0026
  than: 0.0025
  official: 0.0025
  ohio: 0.0024
  loan: 0.0024
  seaport: 0.0023
  prohibit: 0.0023

Topic 5:
  open: 0.0044
  russia: 0.0029
 

...

/Users/varjuzoltan/Library/Caches/pypoetry/virtualenvs/chronowords-UWSi7A_J-py3.12/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(



Top topics:

Topic 0:
  natural: 0.0034
  housing: 0.0032
  very: 0.0031
  rule: 0.0031
  four: 0.0030
  national: 0.0030
  enough: 0.0029
  never: 0.0028
  receive: 0.0028
  abuse: 0.0027

Topic 1:
  foreign: 0.0033
  belong: 0.0032
  objective: 0.0032
  liberal: 0.0031
  wage: 0.0031
  join: 0.0031
  restore: 0.0030
  training: 0.0028
  will: 0.0028
  lot: 0.0028

Topic 2:
  aim: 0.0037
  serve: 0.0033
  remove: 0.0033
  nicaragua: 0.0033
  quarter: 0.0032
  demonstrate: 0.0032
  write: 0.0031
  100: 0.0030
  transportation: 0.0030
  which: 0.0029

Topic 3:
  assistance: 0.0039
  bush: 0.0036
  current: 0.0032
  exchange: 0.0031
  capability: 0.0030
  afghanistan: 0.0030
  key: 0.0030
  you: 0.0029
  when: 0.0028
  law: 0.0028

Topic 4:
  learn: 0.0038
  fast: 0.0036
  review: 0.0035
  reduction: 0.0033
  especially: 0.0032
  dignity: 0.0032
  place: 0.0031
  teacher: 0.0029
  somebody: 0.0028
  would: 0.0028

Topic 5:
  why: 0.0033
  important: 0.0033
  address: 0.0032
  rest: 0.00

/Users/varjuzoltan/Library/Caches/pypoetry/virtualenvs/chronowords-UWSi7A_J-py3.12/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(



Top topics:

Topic 0:
  fire: 0.0037
  organization: 0.0034
  source: 0.0033
  force: 0.0032
  accountable: 0.0030
  lift: 0.0030
  plan: 0.0030
  each: 0.0030
  urge: 0.0029
  stability: 0.0029

Topic 1:
  development: 0.0040
  shape: 0.0035
  actually: 0.0034
  immediately: 0.0033
  which: 0.0031
  eliminate: 0.0031
  stem: 0.0031
  consequence: 0.0030
  fuel: 0.0029
  national: 0.0029

Topic 2:
  research: 0.0036
  official: 0.0036
  open: 0.0035
  america: 0.0033
  class="s1">i: 0.0032
  civil: 0.0032
  religious: 0.0031
  allow: 0.0030
  additional: 0.0029
  spending: 0.0029

Topic 3:
  republicans: 0.0041
  person: 0.0033
  use: 0.0031
  mayor: 0.0031
  follow: 0.0030
  event: 0.0030
  sanction: 0.0030
  commit: 0.0029
  set: 0.0028
  bear: 0.0028

Topic 4:
  refuse: 0.0037
  resource: 0.0035
  choose: 0.0035
  possibility: 0.0033
  difficult: 0.0033
  transition: 0.0031
  stake: 0.0030
  address: 0.0029
  perhaps: 0.0029
  within: 0.0029

Topic 5:
  remove: 0.0037
  industry: 0

/Users/varjuzoltan/Library/Caches/pypoetry/virtualenvs/chronowords-UWSi7A_J-py3.12/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(


In [11]:
def process_topic_evolution(topic_models):
    """Process topic models into evolution chains for visualization.

    Args:
        topic_models: Dictionary mapping periods to TopicModel instances

    Returns:
        List of dicts containing evolution data for visualization

    """
    all_data = []
    # Sort periods chronologically
    periods = sorted(topic_models.keys(), key=lambda x: int(x.split("-")[0]))

    # Initialize chains from first period
    first_period = periods[0]
    first_model = topic_models[first_period]

    # Create initial topic chains
    chains = {}
    for idx, topic in enumerate(first_model.topics):
        chain_id = f"Chain_{idx}"
        top_words = [word for word, _ in topic.words[:3]]
        topic_label = ", ".join(top_words)

        chains[chain_id] = {
            "id": chain_id,
            "current_topic": topic,
            "label": topic_label,
        }

        # Add first period data point
        all_data.append(
            {
                "year": int(first_period.split("-")[0]),
                "period": first_period,
                "chain_id": chain_id,
                "topic_label": topic_label,
                "weight": topic.words[0][1],  # Use weight of top word
                "words": topic_label,
            }
        )

    # Follow alignments through subsequent periods
    for i in range(len(periods) - 1):
        period1, period2 = periods[i], periods[i + 1]
        model1 = topic_models[period1]
        model2 = topic_models[period2]

        # Get aligned topics
        aligned_topics = model1.align_with(model2)

        # Update chains based on alignments
        new_chains = {}
        used_target_topics = set()

        for aligned in aligned_topics:
            # Find chain containing source topic
            source_chain = None
            for chain_id, chain in chains.items():
                if chain["current_topic"].id == aligned.source_topic.id:
                    source_chain = chain
                    break

            if source_chain and aligned.target_topic.id not in used_target_topics:
                chain_id = source_chain["id"]
                top_words = [word for word, _ in aligned.target_topic.words[:3]]
                topic_label = ", ".join(top_words)

                # Add data point for this period
                all_data.append(
                    {
                        "year": int(period2.split("-")[0]),
                        "period": period2,
                        "chain_id": chain_id,
                        "topic_label": source_chain[
                            "label"
                        ],  # Keep original label for continuity
                        "weight": aligned.target_topic.words[0][1],
                        "words": topic_label,  # Current words for tooltip
                    }
                )

                # Update chain
                new_chains[chain_id] = {
                    "id": chain_id,
                    "current_topic": aligned.target_topic,
                    "label": source_chain["label"],
                }
                used_target_topics.add(aligned.target_topic.id)

        chains = new_chains

    return all_data


def plot_topic_trends(topic_models):
    """Create interactive visualization of topic evolution over time.

    Args:
        topic_models: Dictionary mapping periods to TopicModel instances

    Returns:
        altair.Chart: Interactive visualization

    """
    import altair as alt
    import pandas as pd

    # Process data
    all_data = process_topic_evolution(topic_models)
    df = pd.DataFrame(all_data)

    # Create base chart
    base = alt.Chart(df).encode(
        x=alt.X(
            "year:Q",
            title="Year",
            scale=alt.Scale(domain=[df.year.min(), df.year.max()]),
        ),
        color=alt.Color(
            "topic_label:N",
            legend=alt.Legend(title="Topics", orient="bottom", columns=2),
        ),
        tooltip=[
            alt.Tooltip("year:Q", title="Year"),
            alt.Tooltip("words:N", title="Top Words"),
            alt.Tooltip("weight:Q", title="Weight", format=".3f"),
        ],
    )

    # Create line chart with points
    lines = base.mark_line(size=2).encode(
        y=alt.Y("weight:Q", title="Topic Weight"), detail="chain_id:N"
    )

    points = base.mark_circle(size=60).encode(y=alt.Y("weight:Q", title="Topic Weight"))

    # Combine charts
    chart = (
        (lines + points)
        .properties(width=800, height=400, title="Topic Evolution Over Time")
        .interactive()
    )

    # Add selection
    topic_selection = alt.selection_point(fields=["topic_label"], bind="legend")

    chart = chart.add_params(topic_selection).encode(
        opacity=alt.condition(topic_selection, alt.value(1), alt.value(0.2))
    )

    return chart


# Example usage:
topic_models = analyze_topics_by_period(models)
chart = plot_topic_trends(topic_models)
chart
# chart.save('topic_evolution.html')  # Save interactive chart


Analyzing topics for 1775-1799


/Users/varjuzoltan/Library/Caches/pypoetry/virtualenvs/chronowords-UWSi7A_J-py3.12/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(



Top topics:

Topic 0:
  punishment: 0.0029
  reason: 0.0027
  mint: 0.0027
  health: 0.0026
  people: 0.0026
  know: 0.0025
  expect: 0.0025
  liberal: 0.0025
  convenience: 0.0025
  competent: 0.0024

Topic 1:
  surrender: 0.0030
  accelerate: 0.0029
  further: 0.0027
  encouragement: 0.0026
  assist: 0.0026
  humane: 0.0026
  half: 0.0025
  gratitude: 0.0025
  diffuse: 0.0025
  day: 0.0024

Topic 2:
  executive: 0.0029
  service: 0.0027
  benevolence: 0.0027
  month: 0.0027
  offender: 0.0026
  check: 0.0026
  commensurate: 0.0026
  george: 0.0026
  partial: 0.0025
  election: 0.0025

Topic 3:
  report: 0.0032
  limit: 0.0028
  appropriation: 0.0027
  continuance: 0.0027
  universal: 0.0026
  answer: 0.0026
  other: 0.0026
  minister: 0.0026
  river: 0.0025
  reside: 0.0024

Topic 4:
  ambition: 0.0030
  renew: 0.0027
  island: 0.0026
  forbid: 0.0026
  mark: 0.0025
  board: 0.0025
  some: 0.0025
  character: 0.0025
  facilitate: 0.0025
  fishery: 0.0024

Topic 5:
  implore: 0.0029


/Users/varjuzoltan/Library/Caches/pypoetry/virtualenvs/chronowords-UWSi7A_J-py3.12/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(



Top topics:

Topic 0:
  rather: 0.0027
  late: 0.0027
  derive: 0.0026
  previous: 0.0026
  line: 0.0026
  current: 0.0026
  station: 0.0025
  order: 0.0024
  continue: 0.0024
  themselves: 0.0024

Topic 1:
  impose: 0.0038
  man: 0.0031
  island: 0.0027
  april: 0.0027
  south: 0.0026
  individual: 0.0026
  loan: 0.0025
  national: 0.0024
  majesty: 0.0024
  purchase: 0.0023

Topic 2:
  resource: 0.0033
  public: 0.0033
  loss: 0.0032
  scene: 0.0028
  involve: 0.0028
  question: 0.0027
  desire: 0.0027
  neither: 0.0027
  let: 0.0027
  proposition: 0.0027

Topic 3:
  open: 0.0043
  accrue: 0.0028
  regulate: 0.0027
  trust: 0.0026
  sustain: 0.0026
  suppression: 0.0025
  same: 0.0025
  dollar: 0.0025
  convention: 0.0025
  well: 0.0024

Topic 4:
  pass: 0.0040
  pay: 0.0031
  similar: 0.0030
  earth: 0.0029
  promote: 0.0028
  coast: 0.0028
  participation: 0.0028
  attention: 0.0027
  revolution: 0.0026
  forbid: 0.0025

Topic 5:
  delay: 0.0028
  prayer: 0.0028
  these: 0.0028
  

...

/Users/varjuzoltan/Library/Caches/pypoetry/virtualenvs/chronowords-UWSi7A_J-py3.12/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(



Top topics:

Topic 0:
  n’t: 0.0032
  refuse: 0.0030
  principle: 0.0030
  whether: 0.0029
  lot: 0.0029
  week: 0.0028
  measure: 0.0028
  above: 0.0027
  important: 0.0027
  represent: 0.0027

Topic 1:
  class="s1">i: 0.0035
  official: 0.0035
  additional: 0.0032
  research: 0.0032
  open: 0.0031
  america: 0.0031
  civil: 0.0030
  white: 0.0029
  religious: 0.0029
  allow: 0.0029

Topic 2:
  republicans: 0.0035
  use: 0.0033
  commit: 0.0033
  mayor: 0.0032
  sanction: 0.0032
  bear: 0.0029
  mind: 0.0028
  save: 0.0028
  university: 0.0028
  threat: 0.0028

Topic 3:
  fake: 0.0037
  effective: 0.0035
  speech: 0.0034
  choose: 0.0033
  stem: 0.0033
  fully: 0.0032
  liberty: 0.0031
  stake: 0.0031
  science: 0.0029
  rebuild: 0.0029

Topic 4:
  fire: 0.0044
  urge: 0.0036
  african: 0.0033
  law: 0.0033
  organization: 0.0030
  brave: 0.0030
  win: 0.0030
  deep: 0.0030
  anyone: 0.0029
  private: 0.0028

Topic 5:
  remove: 0.0040
  industry: 0.0034
  organization: 0.0033
  testi

/Users/varjuzoltan/Library/Caches/pypoetry/virtualenvs/chronowords-UWSi7A_J-py3.12/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(



Top topics:

Topic 0:
  ’ll: 0.0037
  california: 0.0033
  good: 0.0032
  need: 0.0032
  bold: 0.0031
  team: 0.0031
  tonight: 0.0030
  watch: 0.0030
  pillar: 0.0030
  each: 0.0029

Topic 1:
  our: 0.0037
  toxic: 0.0035
  economy: 0.0033
  try: 0.0032
  inform: 0.0031
  exceptional: 0.0031
  badly: 0.0029
  technology: 0.0029
  voice: 0.0028
  trump: 0.0026

Topic 2:
  people: 0.0040
  2025: 0.0040
  never: 0.0033
  oil: 0.0032
  plant: 0.0030
  purpose: 0.0029
  policy: 0.0029
  more: 0.0029
  hurricane: 0.0028
  delaware: 0.0028

Topic 3:
  mental: 0.0037
  flourish: 0.0034
  wealth: 0.0033
  presidential: 0.0032
  stars: 0.0032
  engineer: 0.0031
  presence: 0.0030
  work: 0.0030
  happen: 0.0030
  liquid: 0.0029

Topic 4:
  possibility: 0.0037
  beginning: 0.0035
  abuse: 0.0033
  police: 0.0031
  address: 0.0031
  peril: 0.0030
  start: 0.0030
  influence: 0.0030
  bullet: 0.0029
  panama: 0.0029

Topic 5:
  expectation: 0.0040
  cost: 0.0038
  some: 0.0038
  los: 0.0032
  mod

alt.LayerChart(...)